In [1]:

from heapq import heappush, heappop, heapify
from collections import defaultdict

class DeletableMinMaxPQ():
    def __init__(self, arr=[], isSet=False): # isSet: if True, then it is a set else it is a multiset
        self.minH = []
        self.maxH = []
        self.HC = defaultdict(int)
        self.size = 0
        self.isSet = isSet
        for x in arr:
            if self.isSet and self.HC[x] > 0:
                continue
            self.minH.append(x)
            self.maxH.append(-x)
            self.HC[x] += 1
            self.size += 1
        heapify(self.minH)
        heapify(self.maxH)

    def add(self, x: int) -> bool:
        if self.isSet and self.HC[x] > 0:
            return False
        heappush(self.minH, x)
        heappush(self.maxH, -x)
        self.HC[x] += 1
        self.size += 1
        return True

    def min(self) -> int:
        assert self.size > 0
        t = self.minH[0]
        while not self.HC[t]:
            heappop(self.minH)
            t = self.minH[0]
        return t

    def extractMin(self) -> int:
        assert self.size > 0
        t = heappop(self.minH)
        while not self.HC[t]:
            t = heappop(self.minH)
        self.HC[t] -= 1
        self.size -= 1
        return t
    
    def max(self) -> int:
        assert self.size > 0
        t = self.maxH[0]
        while not self.HC[-t]:
            heappop(self.maxH)
            t = self.maxH[0]
        return -t
    
    def extractMax(self) -> int:
        assert self.size > 0
        t = -heappop(self.maxH)
        while not self.HC[t]:
            t = -heappop(self.maxH)
        self.HC[t] -= 1
        self.size -= 1
        return t
    
    def remove(self, x) -> bool:
        if self.HC[x] > 0:
            self.HC[x] -= 1
            self.size -= 1
            return True
        return False
    
    def __contains__(self, x: int) -> bool:
        return self.HC[x] > 0
    
    def __len__(self):
        return self.size
    
    def __bool__(self):
        return self.size > 0
    
    def __repr__(self):
        items = []
        for k in self.HC:
            items.extend([k]*self.HC[k])
        return str(sorted(items))

In [2]:
s = DeletableMinMaxPQ([1,2,3,4,5,6,7,8,9,10])

In [3]:
s

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [4]:
s.add(3)

True

In [5]:
s

[1, 2, 3, 3, 4, 5, 6, 7, 8, 9, 10]

In [6]:
5 in s

True

In [7]:
s.remove(5)

True

In [8]:
s.remove(5)

False

In [9]:
s

[1, 2, 3, 3, 4, 6, 7, 8, 9, 10]

In [10]:
5 in s

False

In [11]:
len(s)

10

In [12]:
s.remove(3)

True

In [13]:
s.remove(3)

True

In [14]:
len(s)

8

In [15]:
bool(s)

True

In [19]:
s.extractMax()

10

In [20]:
s

[1, 2, 4, 6, 7, 8, 9]

In [21]:
len(s)

7

In [22]:
s.max()

9

In [23]:
s.max()

9

In [24]:
10 in s

False

In [25]:
s.remove(10)

False

In [26]:
s.remove(9)

True

In [27]:
s.max()

8

In [28]:
s.extractMin()

1

In [29]:
1 in s

False

In [30]:
s.remove(2)

True

In [31]:
len(s)

4

In [32]:
s

[4, 6, 7, 8]

In [33]:
s.remove(6)

True

In [34]:
s

[4, 7, 8]

In [35]:
s.extractMin()

4

In [36]:
s

[7, 8]

In [37]:
s.extractMin()

7

In [39]:
s.extractMin()

8

In [40]:
s.extractMin()

AssertionError: 

In [29]:

class FenwickTree:
    def __init__(self, x):
        """transform list into BIT"""
        self.arr = x
        x = self.bit = x[:]
        for i in range(len(x)):
            j = i | (i + 1)
            if j < len(x):
                x[j] += x[i]

    def update(self, idx, x):
        """updates bit[idx] += x"""
        self.arr[idx] += x
        while idx < len(self.bit):
            self.bit[idx] += x
            idx |= idx + 1

    def __getitem__(self, idx):
        return self.arr[idx]
    
    def __setitem__(self, idx, x):
        """updates bit[idx] = x"""
        self.update(idx, x - self.arr[idx])
    
    def __iadd__(self, idx, x):
        """updates bit[idx] += x"""
        self.update(idx, x)


    def sum(self, end):
        """calc sum from [0, end) (zero based)"""
        x = 0
        while end > 0:
            x += self.bit[end - 1]
            end &= end - 1
        return x
    
    def query(self, begin, end):
        """calc sum from [begin, end) (zero based)"""
        if begin >= end:
            return 0
        return self.sum(end) - self.sum(begin)

    def findkth(self, k):
        """Find largest idx such that sum from [0, idx) <= k"""
        idx = -1
        for d in reversed(range(len(self.bit).bit_length())):
            right_idx = idx + (1 << d)
            if right_idx < len(self.bit) and k >= self.bit[right_idx]:
                idx = right_idx
                k -= self.bit[idx]
        return idx + 1
    
    def __repr__(self):
        return "BIT({})".format(self.arr)


class RangeUpdatePointQuery:
    def __init__(self, arr):
        self.arr = arr
        self.bit = FenwickTree([0] * (len(arr) + 1))
    
    def update(self, l, r, x):
        """updates arr[l:r] += x"""
        self.bit.update(l, x)
        self.bit.update(r, -x)
    
    def __getitem__(self, idx):
        return self.arr[idx] + self.bit.sum(idx+1)
    
    def __repr__(self):
        return "RUPQ({})".format([self[i] for i in range(len(self.arr))])

In [30]:
rupq = RangeUpdatePointQuery([1,2,3,4,5,6,7,8,9,10])

In [34]:
rupq.update(3, 5, 100)
rupq

RUPQ([101, 102, 103, 404, 405, 6, 7, 8, 9, 10])